# Prerequisite import

In [ ]:
import gc
import pandas as pd
from collections import OrderedDict
import json


In [ ]:
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader

In [ ]:
from ignite.engine import *
from ignite.handlers import *
from ignite.metrics import *
from ignite.utils import *
from ignite.contrib.metrics.regression import *
from ignite.contrib.metrics import *

def eval_step(engine, batch):
    return batch

default_evaluator = Engine(eval_step)

param_tensor = torch.zeros([1], requires_grad=True)
default_optimizer = torch.optim.SGD([param_tensor], lr=0.1)

def get_default_trainer():

    def train_step(engine, batch):
        return batch

    return Engine(train_step)

# create default model for doctests

default_model = nn.Sequential(OrderedDict([
    ('base', nn.Linear(4, 2)),
    ('fc', nn.Linear(2, 1))
]))

manual_seed(666)

# Data Load

In [ ]:
import pickle



def load_obj(name ):
    with open('./' + name + '.pkl', 'rb') as f:
        return pickle.load(f)

users = pickle.load(open("users_new.pkl", 'rb'))
reviews = pickle.load(open("reviews.pkl", 'rb'))

# Data analysis

In [ ]:
label_1 = 0
for item in users:
    view_seq = [[0, 0, 0, 0, 0, 0, 0, 0, 0] for i in range(124)]
    labels = 0
    year = 2012
    # print(item)
    if len(users[item]["reviews"]) == 0:
        continue
    for review in users[item]["reviews"]:
        view = reviews[review].copy()
        # print(view["date"])
        view["date"] = view["date"].split('-')
        view_seq[(int(view["date"][0]) - year) * 12 + int(view["date"][1]) - 1][0] += 1
        view_seq[(int(view["date"][0]) - year) * 12 + int(view["date"][1]) - 1][1] += view["stars"]
        view_seq[(int(view["date"][0]) - year) * 12 + int(view["date"][1]) - 1][2] += view["funny"]
        view_seq[(int(view["date"][0]) - year) * 12 + int(view["date"][1]) - 1][3] += view["useful"]
        view_seq[(int(view["date"][0]) - year) * 12 + int(view["date"][1]) - 1][4] += view["cool"]
        view_seq[(int(view["date"][0]) - year) * 12 + int(view["date"][1]) - 1][5] += view["sentiment_score"]['neg']
        view_seq[(int(view["date"][0]) - year) * 12 + int(view["date"][1]) - 1][6] += view["sentiment_score"]['neu']
        view_seq[(int(view["date"][0]) - year) * 12 + int(view["date"][1]) - 1][7] += view["sentiment_score"]['pos']
        view_seq[(int(view["date"][0]) - year) * 12 + int(view["date"][1]) - 1][8] += view["sentiment_vader"]
    for i in range(len(view_seq)):
        if view_seq[i][0] != 0:
            view_seq[i][1] /= view_seq[i][0]
            view_seq[i][2] /= view_seq[i][0]
            view_seq[i][3] /= view_seq[i][0]
            view_seq[i][4] /= view_seq[i][0]
            view_seq[i][5] /= view_seq[i][0]
            view_seq[i][6] /= view_seq[i][0]
            view_seq[i][7] /= view_seq[i][0]
            view_seq[i][8] /= view_seq[i][0]

    sum_score = 0
    for i in range(len(view_seq)-36, len(view_seq)):
        sum_score += view_seq[i][0] 
    if sum_score > 0:       
        labels = 1
        label_1 += 1
    users[item]["view_seq"] = view_seq
    users[item]["labels"] = labels
print(label_1)

In [ ]:
for item in users:
    label_churn = 0
    if len(users[item]['friends_valid']) == 0:
        users[item]['friend_churn'] = 1
        continue
    for f in users[item]['friends_valid']:
        if users[f]['labels'] == 0:
            label_churn += 1
    users[item]['friend_churn'] = label_churn / len(users[item]['friends_valid'])

In [ ]:
for item in users:
  print(users[item]["view_seq"][:])
  for i in range(len(users[item]['view_seq'])):
    if users[item]['view_seq'][i][0] > 0:
      print(i)
  print(users[item]['labels'])
  #for i in range(len(users[item]['labels'])):
    # if users[item]['labels'][i] > 0:
    #   print(i)
  print(users[item]['since'])
  break

In [ ]:
import numpy as np
for item in users: 
  features = np.array(users[item]["view_seq"])
  features = features[:88].sum(axis = 0)
  users[item]['view_features'] = features


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def FeaturesProcess(users, reviews):
  seq = {}
  for item in users:
    user_features = np.array([])
    user_features = np.append(user_features, users[item]['review_count'])
    user_features = np.append(user_features, users[item]['fans'])
    user_features = np.append(user_features, len(users[item]['friends']))
    user_features = np.append(user_features, users[item]['friend_churn'])


    user_features = np.append(user_features, users[item]['comments'])
    user_features = np.append(user_features, users[item]['compliment'])
    user_features = np.append(user_features, users[item]['stars'])
    

    user_features = np.hstack((user_features,users[item]['view_features']))
    user_features = np.hstack((user_features,users[item]['labels']))
    seq[item] = user_features
  return seq

features_seq = FeaturesProcess(users, reviews)

In [ ]:
for item in features_seq:
  print(features_seq[item])
  print(features_seq[item].shape)
  break
  

# LR Mode

In [ ]:
import random
def features_org(seq):

    train_Xs = []
    train_Ys = []
    train_Us = []
    test_Xs = []
    test_Ys = []
    test_Us = []
    train_len0 = 0
    train_len1 = 0
    test_len0 = 0
    test_len1 = 0
    count = 0

    review_count = 0

    for item in seq:
        features = seq[item]
        # if random.random() > 0.1:
        #     continue

        if random.random() < 0.9:
            if features[-1] == 0 and train_len0 > 1.1 * train_len1:
                continue

            review_count += users[item]['review_count_valid']

            if features[-1] == 1:
                train_len1 += 1
                count+=1
            else:
                train_len0 += 1
            train_Us.append(features[:13])
            train_Xs.append(features[13:-1])
            train_Ys.append([features[-1]])
            
        else:
            if features[-1] == 0 and test_len0 > 1.1 * test_len1:
                continue

            review_count += users[item]['review_count_valid']

            if features[-1] == 1:
                test_len1 += 1
                count+=1 
            else:
                test_len0 += 1
            test_Us.append(features[:13])
            test_Xs.append(features[13:-1])
            test_Ys.append([features[-1]])


    print(count)
    print(review_count)       
    return (torch.tensor(train_Us), torch.tensor(train_Xs), torch.tensor(train_Ys), torch.tensor(test_Us), torch.tensor(test_Xs), torch.tensor(test_Ys))
(train_Us, train_Xs, train_Ys, test_Us, test_Xs, test_Ys) = features_org(features_seq)

print(len(test_Us))
print(len(test_Xs))
print(len(test_Ys))
print(len(train_Us))
print(len(train_Xs))
print(len(train_Ys))
print(test_Xs.shape)



In [ ]:
print(len(test_Us))
print(len(test_Xs))
print(len(test_Ys))
print(len(train_Us))
print(len(train_Xs))
print(len(train_Ys))
print(test_Us.shape)

s = 0
for i in train_Ys:
    if i == 1:
        s+=1
print(s)

s = 0
for i in test_Ys:
    if i == 1:
        s+=1
print(s)

In [ ]:
print(len(reviews))

In [ ]:


class Seq2SeqDataset(Dataset):   
    def __init__(self, U, X, Y):
        super().__init__()
        
        # check inputs
        assert len(X)==len(Y) and len(X) == len(U) and len(X)>0
        
        self.Us = U
        self.Xs = X
        self.Ys = Y

        self.num_example = len(U)
        
    def __getitem__(self, idx):
        return self.Us[idx], self.Xs[idx], self.Ys[idx]
    
    def __len__(self):
        return self.num_example



In [ ]:
class LR(nn.Module):

    def __init__(self, input_dim, output_dim, user_dim, view_dim, pre, mode = 0):
        super().__init__()

        self.mode = mode
        self.pre = pre

        
        self.norm1 = nn.BatchNorm1d(view_dim)
        self.norm2 = nn.BatchNorm1d(user_dim)
        self.acti = nn.ELU()
        self.pre = nn.Sigmoid()
        self.fine = nn.Softmax(dim = 1)
        self.drop = nn.Dropout2d(0.2)

        self.linear1 = nn.Linear(input_dim, 32)
        self.linear2 = nn.Linear(32, 64)
        self.linear3 = nn.Linear(64, 48)
        self.linear4 = nn.Linear(48, 16)
        self.linear5 = nn.Linear(16, 8)

        self.linear6 = nn.Linear(8, 2)
        self.linear7 = nn.Linear(8, 1)

        self.linear8 = nn.Linear(64, 128)
        self.linear9 = nn.Linear(128, 256)
        self.linear10 = nn.Linear(256, 256)
        self.linear11 = nn.Linear(256, 192)
        self.linear12 = nn.Linear(192, 192)
        self.linear13 = nn.Linear(192, 128)
        self.linear14 = nn.Linear(128, 96)
        self.linear15 = nn.Linear(96, 64)
        self.linear16 = nn.Linear(64, 64)


    def forward(self, X, U):
        X = self.norm1(X)
        U = self.norm2(U)
        
        if self.mode  == 0:
            input = torch.cat([X,U],axis = 1)
        elif self.mode  == 1:
            input = X
        else:
            input = U

        hidden = self.linear1(input)
        hidden = self.linear2(hidden)

        hidden = self.acti(hidden)
        hidden = self.linear8(hidden)
        hidden = self.drop(hidden)
        hidden = self.linear9(hidden)
        hidden = self.acti(hidden)
        hidden = self.linear10(hidden)
        hidden = self.drop(hidden)
        hidden = self.linear11(hidden)
        hidden = self.acti(hidden)
        hidden = self.linear12(hidden)
        hidden = self.drop(hidden)
        hidden = self.linear13(hidden)
        hidden = self.drop(hidden)
        hidden = self.linear14(hidden)
        hidden = self.acti(hidden)
        hidden = self.linear15(hidden)
        hidden = self.drop(hidden)
        hidden = self.linear16(hidden)
        hidden = self.acti(hidden)

        hidden = self.linear3(hidden)
        hidden = self.linear4(hidden)
        hidden = self.acti(hidden)
        hidden = self.linear5(hidden)
        hidden = self.acti(hidden)

        if True:
            y_hat = self.linear7(hidden)
            y_hat = self.pre(y_hat)
        else:
            y_hat = self.linear6(hidden)
            y_hat = self.fine(y_hat)

        return y_hat


In [ ]:
def loss_count_1(pred, true):
    count = 0
    for i in range(len(pred)):
        if (pred[i][0] > pred[i][1] and true[i][0] == 0) or (pred[i][1] >= pred[i][0] and true[i][0] == 1):
            count += 1
    return count / len(true)

def loss_count(pred, true):
    count = 0
    for i in range(len(pred)):  
        if (pred[i][0] >= 0.5 and true[i][0] == 1) or (pred[i][0] < 0.5 and true[i][0] == 0):
            count += 1
    return count / len(true)



In [ ]:

input_dim = 22
output_dim = 2
batch_size = 64
user_dim = 13
view_dim = 9
learning_rate = 0.05
LRpre = True

loss_fn = nn.MSELoss()# nn.BCELoss()
loss_fn_new = nn.CrossEntropyLoss()

train_Xs1 = train_Xs[:]
train_Us1 = train_Us[:]
train_Ys1 = train_Ys[:]

test_Xs1 = test_Xs[:]
test_Us1 = test_Us[:]
test_Ys1 = test_Ys[:]

train_dataset = Seq2SeqDataset(train_Us1, train_Xs1, train_Ys1)
test_dataset = Seq2SeqDataset(test_Us1, test_Xs1, test_Ys1)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
eval_loader  = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

model = LR(input_dim, output_dim, user_dim, view_dim, LRpre, 0)
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
model.double()


roc_auc = ROC_AUC()
roc_auc.attach(default_evaluator, 'roc_auc')

In [ ]:
history = []
# model = model.double().cuda()

best_score = 0
metric = []

epoch = 10
for epoch in range(epoch):
    # training loop
    model.train()
    losses = []
    for i, batch in enumerate(train_loader):
        # forward pass
        U, X, Y = batch
        U, X, Y = U, X, Y
        y_hat = model(X, U)

        if LRpre:
            loss = loss_fn(y_hat.double(), Y.double())
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            losses.append(loss.detach())
        else:
            loss = loss_fn_new(y_hat.double().squeeze(0), torch.cat([Y[i] for i in range(len(Y))], 0).long())
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            losses.append(loss.detach())

    # calculate epoch loss and MAPE
    train_loss = torch.mean(torch.tensor(losses)).item()

    # evaluation loop
    model.eval()
    losses = []
    loss_num = []
    auc_list = []
    with torch.no_grad():
        for batch in eval_loader:
            # forward pass
            U, X, Y = batch
            U, X, Y = U, X, Y
            y_hat = model(X, U)
            if LRpre:
                loss = loss_fn(y_hat.double(), Y.double())
                loss_num.append(loss_count(y_hat, Y))
                losses.append(loss.detach())
                #state = default_evaluator.run([[model(X, U), Y]])
                #print(state.metrics['roc_auc'])
                auc_list.append(default_evaluator.run([[y_hat, Y]]).metrics['roc_auc'])
            else:
                loss = loss_fn_new(y_hat.double().squeeze(0), torch.cat([Y[i] for i in range(len(Y))], 0).long())
                loss_num.append(loss_count_1(y_hat.double().squeeze(0), Y.double()))
                losses.append(loss.detach())


                

        # calculate evaluateion loss and MAPE for current epoch
        eval_loss = torch.mean(torch.tensor(losses)).item()
        count_loss = torch.mean(torch.tensor(loss_num)).item()
        if LRpre:
            auc_loss = torch.mean(torch.tensor(auc_list)).item()
        else:
            auc_loss = 0

        if count_loss > best_score:
            best_score = count_loss
            best_ckpt = model.state_dict()
    # record training curves and metrics
    metrics = {
        'epoch'   : epoch + 1,
        'train_loss': train_loss,
        'eval_loss' : eval_loss,
        'pred' : count_loss,
        'auc' : auc_loss
    }
    history.append(metrics)
    metric.append(metrics)

    print(metrics)

print(best_score)
torch.save({
    'model_state_dict': best_ckpt,
}, "./LR_checkpoint.ckpt")


jsObj = json.dumps(metric)
fileObject = open('jsonFileLR.json', 'w')
fileObject.write(jsObj)
fileObject.close()

# Semi-RNN Model

In [ ]:
# for item in users:
#   view_seq = []
#   for i in range(88):
#     if sum(users[item]["view_seq"][i]) != 0:
#         users[item]["view_seq"][i].append(i)
#         view_seq.append(users[item]["view_seq"][i])
#   users[item]["view_seq_valid"] = view_seq

In [ ]:
import random
def features_org(users, features_seq):

    train_Xs = []
    train_Ys = []
    train_Us = []
    test_Xs = []
    test_Ys = []
    test_Us = []
    train_len0 = 0
    train_len1 = 0
    test_len0 = 0
    test_len1 = 0
    count = 0

    review_count = 0

    for item in users:
        label = users[item]['labels']
        # if random.random() > 0.1:
        #     continue

        if random.random() < 0.9:
            if label == 0 and train_len0 > 1.1 * train_len1:
                continue

            review_count += users[item]['review_count_valid']

            if label == 1:
                train_len1 += 1
                count+=1
            else:
                train_len0 += 1

            train_Us.append(features_seq[item][:13])
            train_Xs.append(users[item]['view_seq'][:88])
            train_Ys.append(label)
            
        else:
            if label == 0 and test_len0 > 1.1 * test_len1:
                continue

            review_count += users[item]['review_count_valid']

            if label == 1:
                test_len1 += 1
                count+=1 
            else:
                test_len0 += 1

            test_Us.append(features_seq[item][:13])
            test_Xs.append(users[item]['view_seq'][:88])
            test_Ys.append(label)


    print(count)
    print(review_count)       
    return (torch.tensor(train_Us), torch.tensor(train_Xs), torch.tensor(train_Ys), torch.tensor(test_Us), torch.tensor(test_Xs), torch.tensor(test_Ys))
(train_Us, train_Xs, train_Ys, test_Us, test_Xs, test_Ys) = features_org(users, features_seq)

print(len(test_Us))
print(len(test_Xs))
print(len(test_Ys))
print(len(train_Us))
print(len(train_Xs))
print(len(train_Ys))
print(test_Xs.shape)


In [ ]:
class RNNmodel(nn.Module):

    def __init__(self, enc_input_dim, output_dim, user_dim, hidden_dim, layers, bid, user):
        super().__init__()
        # encoder
        self.norm1 = nn.BatchNorm1d(enc_input_dim)
        self.norm2 = nn.BatchNorm1d(user_dim)
        self.norm3 = nn.BatchNorm1d(hidden_dim)
        
        self.enc_rnn = nn.GRU(
            input_size=enc_input_dim,
            hidden_size=hidden_dim,
            num_layers= layers,
            batch_first=True,
            bidirectional=bid
        )

        self.bid = bid
        self.user = user
        if bid:
            linear_dim = hidden_dim * 2 * layers + user
            self.norm3 = nn.BatchNorm1d(hidden_dim * 2 * layers)
        else:
            linear_dim = hidden_dim * layers + user
            self.norm3 = nn.BatchNorm1d(hidden_dim * layers)

        # user LR
        self.linear7 = nn.Linear(user_dim, 32)
        self.linear8 = nn.Linear(32, 64)
        self.linear9 = nn.Linear(64, 128)
        self.linear10 = nn.Linear(128, 48)
        self.linear11 = nn.Linear(48, 32)
        self.linear12 = nn.Linear(32, user_dim)


        # linear predictor
        self.linear1 = nn.Linear(linear_dim, 512)
        self.linear2 = nn.Linear(512, 256)
        self.linear3 = nn.Linear(256, 128)
        self.linear4 = nn.Linear(128, 64)
        self.linear5 = nn.Linear(64, 32)
        self.linear6 = nn.Linear(32, output_dim)



        self.acti = nn.ELU()
        self.drop = nn.Dropout2d(0.2)
        self.pre = nn.Sigmoid()


    def forward(self, X, U):
        X = self.norm1(X.transpose(1, 2)).transpose(1, 2)
        U = self.norm2(U)
        output, hidden = self.enc_rnn(X)  # encoding
        
        U = self.linear12(self.linear11(self.linear10(self.acti(self.linear9(self.acti(self.linear8(self.acti(self.linear7(U)))))))))
        
        hidden = torch.cat([hidden[i] for i in range(len(hidden))], 1).unsqueeze(0)

        hidden = self.norm3(hidden.squeeze(0)).unsqueeze(0)
        
        if self.user > 0:
            hidden = torch.cat([hidden, U.unsqueeze(0)], 2)

        # print(hidden)
        y_hat = self.linear6(self.linear5(self.linear4(self.acti(self.linear3(self.acti(self.linear2(self.acti(self.linear1(hidden)))))))))  # predicting
        y_hat = self.pre(y_hat)
  
        return y_hat

In [ ]:

input_dim = 9
output_dim = 1
hidden_dim = 48
batch_size = 32
user_dim = 13
learning_rate = 0.1
loss_fn = nn.BCELoss()# nn.BCELoss()


train_Xs1 = train_Xs[:]
train_Us1 = train_Us[:]
train_Ys1 = train_Ys[:]

test_Xs1 = test_Xs[:]
test_Us1 = test_Us[:]
test_Ys1 = test_Ys[:]

train_dataset = Seq2SeqDataset(train_Us1, train_Xs1, train_Ys1)
test_dataset = Seq2SeqDataset(test_Us1, test_Xs1, test_Ys1)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
eval_loader  = DataLoader(test_dataset, batch_size=batch_size)

In [ ]:

newmodel = RNNmodel(input_dim, output_dim, user_dim, hidden_dim, 10, True, len(train_Us[0]))
newmodel = newmodel.double().cuda()
optimizer = torch.optim.Adam(newmodel.parameters(), lr=learning_rate)

In [ ]:
train_Xs1.unsqueeze(0).shape

In [ ]:
def loss_count_1(pred, true):
    count = 0
    for i in range(len(pred)):
        if (pred[i][0] > pred[i][1] and true[i][0] == 0) or (pred[i][1] >= pred[i][0] and true[i][0] == 1):
            count += 1
    return count / len(true)

In [ ]:
history = []
# model = model.double().cuda()

best_score = 0
metric = []
epoch = 2
for epoch in range(epoch):
    # training loop
    newmodel.train()
    losses = []
    for i, batch in enumerate(train_loader):
        # forward pass
        U, X, Y = batch
        U, X, Y = U.double().cuda(), X.double().cuda(), Y.double().cuda()
        y_hat = newmodel(X, U)

        # loss = loss_fn(y_hat.double().squeeze(0), Y.double())
        loss = loss_fn(y_hat.double().squeeze(0), Y.reshape(-1,1).double())

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        losses.append(loss.detach())

    # calculate epoch loss and MAPE
    train_loss = torch.mean(torch.tensor(losses)).item()

    # evaluation loop
    newmodel.eval()
    losses = []
    loss_num = []
    with torch.no_grad():
        for batch in eval_loader:
            # forward pass
            U, X, Y = batch
            U, X, Y = U.double().cuda(), X.double().cuda(), Y.double().cuda()
            y_hat = newmodel(X, U)

            loss = loss_fn(y_hat.double().squeeze(0), Y.reshape(-1,1).double())


            loss_num.append(loss_count(y_hat.double().squeeze(0), Y.reshape(-1,1).double()))
            losses.append(loss.detach())



        # calculate evaluateion loss and MAPE for current epoch
        eval_loss = torch.mean(torch.tensor(losses)).item()
        count_loss = torch.mean(torch.tensor(loss_num)).item()
        
        if count_loss > best_score:
            best_score = count_loss
            best_ckpt = newmodel.state_dict()
    # record training curves and metrics
    metrics = {
        'epoch'   : epoch + 1,
        'train_loss': train_loss,
        'eval_loss' : eval_loss,
        'pred' : count_loss
    }
    history.append(metrics)
    metric.append(metrics)

    print(metrics)

print(best_score)
torch.save({
    'model_state_dict': best_ckpt,
}, "./checkpoint.ckpt")


jsObj = json.dumps(item)
fileObject = open('jsonFile.json', 'w')
fileObject.write(jsObj)
fileObject.close()

